# Applying BlueFog on Deep Learning problem(High Level API Introduction)

All previous sections we focused on the low-level API in the BlueFog, which is great for flexible algorithm design and research. Give a quick summary here:

- Basic static topology property and its manipulation
- Collective communication such as broadcast and allreduce
- Topology based Neighborhood communication such as neighbor_allreduce.
- Blocking versus non-blocking operation
- Dynamic topology and its corresponding
- Asynchronous operation through window object
- Examine numerous algorithms and their performance under different scenarios
- etc.

However, it can be boilerplate if you want to apply one certain algorithm on different tasks. Further, it is also tricky to write a efficiency code combining above mentioned concepts. This becomes even worse in the deep learning problem. Backpropagation property of neural network makes that the gradient can be efficiently calculated. Backpropagation also implied that the gradient is calculated (approximately) in layer-by-layer style, in contrast to one global stochastic (sub-)gradient we encountered in the optimization. Further, this layer-wise computation provides a great opportunity to overlap the communication and comptuation for minimizing the trainning time, which is a crucial. Clearly, writing the code to address these concern correctly and efficient is not easy. Hence, BlueFog further provides the high level APIs, which can be directly applied on the `torch.Optimizer` directly. 

In this section, we will focus in applying high level APIs of BlueFog on Deep Learning problem, mainly the decentralized trainning task.
Before we demystify how we implement the High-Level API in BlueFog, let's see the example of using them to write distributed trainning of ResNet-18 model over CIFAR-10 easily. (*Note, although this example is relative small, it still can be time-consuming and drain tons of computation resources if you want to train them on CPUs.*)


In [1]:
import torch
import networkx as nx
import ipyparallel as ipp
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
rc = ipp.Client(profile="bluefog")
rc.block=True
rc.ids

[0, 1, 2, 3]

In [4]:
%%px
import torch
import bluefog.torch as bf

bf.init()